<a href="https://colab.research.google.com/github/HarshSinha-1/MachineLearning/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

*Linear Model for Classification [Logistic Regression]**

In [2]:
bd_train=pd.read_csv(r'/content/sample_data/bd_train.csv')
bd_new=pd.read_csv(r'/content/sample_data/bd_test.csv')

In [5]:
bd_train.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment.Tax.Saving.Bond,Home.Loan,Online.Purchase.Amount,Revenue.Grid,gender,region,Investment.in.Commudity,Investment.in.Equity,Investment.in.Derivative,Portfolio.Balance
0,4888,Zero,55-60,Widowed,Retired,Unknown,Own Home,"<10,000, >= 8,000",No,No,...,33.47,12.96,4.99,1,Female,North West,91.85,25.71,95.52,249.82
1,8525,Zero,61-65,Partner,Retired,Retired,Own Home,">=35,000",No,No,...,45.96,28.95,3.99,2,Male,North West,127.65,56.21,89.20,222.27
2,3411,3,31-35,Partner,Professional,Housewife,Own Home,"<25,000, >=22,500",No,No,...,0.00,0.00,0.00,2,Female,North West,4.19,0.00,3.50,17.05
3,692,Zero,51-55,Partner,Secretarial/Admin,Other,Own Home,"<20,000, >=17,500",No,No,...,0.00,0.00,0.00,2,Male,North West,9.59,0.00,7.99,-72.74
4,10726,1,51-55,Partner,Retired,Retired,Own Home,">=35,000",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,15.69,5.83,15.66,-13.98


In [3]:
def children_to_num(col):

    num_col=col.str.replace('Zero','0')
    num_col=num_col.str.replace('4+','4',regex=False)
    num_col=pd.to_numeric(num_col,errors='coerce')

    return num_col

In [4]:
def ab_to_num(col):

    col=col.str.replace('71+','71-71',regex=False)
    k=col.str.split('-',expand=True)

    for i in [0,1]:
        k[i]=pd.to_numeric(k[i],errors='coerce')

    num_col=0.5*(k[0]+k[1])

    return num_col

In [6]:
def fi_to_num(col):

    col=col.replace({'<10,000, >= 8,000':9000, '>=35,000':35000, '<25,000, >=22,500':23750,
       '<20,000, >=17,500':18750, '<12,500, >=10,000':11250, '<30,000, >=27,500':28750,
       '<27,500, >=25,000':26250, '<17,500, >=15,000':16250, '<15,000, >=12,500':13750,
       '<22,500, >=20,000':21250,'< 4,000': 4000, '< 8,000, >= 4,000':6000})
    num_col=pd.to_numeric(col,errors='coerce')

    return num_col

In [7]:
simple_numeric_cols=['year_last_moved','Average.Credit.Card.Transaction', 'Balance.Transfer',
      'Term.Deposit', 'Life.Insurance', 'Medical.Insurance',
      'Average.A.C.Balance', 'Personal.Loan', 'Investment.in.Mutual.Fund',
      'Investment.Tax.Saving.Bond', 'Home.Loan', 'Online.Purchase.Amount','Investment.in.Commudity',
      'Investment.in.Equity', 'Investment.in.Derivative',
      'Portfolio.Balance']

cat_to_dummies_cols=['status' , 'occupation' , 'occupation_partner' , 'home_status', 'self_employed',
'self_employed_partner','TVarea','gender','region']

custom_function_cols={'children':children_to_num,'age_band':ab_to_num,'family_income':fi_to_num}

In [8]:
from google.colab import files
uploaded = files.upload()
import ml_utils as mt

Saving ml_utils.py to ml_utils.py


In [9]:
bd_pipe=mt.DataPipe(simple_num=simple_numeric_cols,
                     cat_to_dummies=cat_to_dummies_cols,
                     custom_func_dict=custom_function_cols)

In [10]:
bd_pipe.fit(bd_train)

In [11]:
x_train=bd_pipe.transform(bd_train)
x_new=bd_pipe.transform(bd_new)
y_train=(bd_train['Revenue.Grid']==1).astype(int)

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
params={'class_weight':['balanced',None],
       'penalty':['l1','l2'] ,# this is not 11 eleven, its L1 [el-one] in lower case
        'C':[.0001,.0005,.001,.005,.01,.05,0.1,2,5,10]}

In [14]:
model=LogisticRegression(solver='liblinear')

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
gs=GridSearchCV(model,
               param_grid=params,
               scoring='roc_auc',
               cv=10,
               n_jobs=-1,
               verbose=20)

In [17]:
gs.fit(x_train,y_train)

Fitting 10 folds for each of 40 candidates, totalling 400 fits


GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'), n_jobs=-1,
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 2,
                               5, 10],
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc', verbose=20)

In [18]:
mt.report(gs.cv_results_)

Model with rank: 1
Mean validation score: 0.955273 (std: 0.006657)
Parameters: {'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 2
Mean validation score: 0.955023 (std: 0.006787)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 3
Mean validation score: 0.955013 (std: 0.006879)
Parameters: {'C': 0.005, 'class_weight': 'balanced', 'penalty': 'l1'}



In [19]:
logr=LogisticRegression(solver='liblinear',**{'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l1'})

In [20]:
logr.fit(x_train,y_train)

LogisticRegression(C=0.05, class_weight='balanced', penalty='l1',
                   solver='liblinear')

In [21]:
(logr.coef_==0).sum() # count of columns for which the weight has been suppressed to 0

52

In [22]:
logr.intercept_ # this w_0

array([0.])

In [23]:
list(zip(x_train.columns,logr.coef_[0]))

[('status_Partner', 0.03262226086048705),
 ('status_Single/Never Married', 0.0),
 ('status_Divorced/Separated', 0.0),
 ('status_Widowed', 0.0),
 ('occupation_Professional', 0.0),
 ('occupation_Retired', 0.0),
 ('occupation_Secretarial/Admin', 0.0),
 ('occupation_Housewife', 0.0),
 ('occupation_Business Manager', 0.0),
 ('occupation_Unknown', 0.0),
 ('occupation_Manual Worker', 0.0),
 ('occupation_Other', 0.0),
 ('occupation_partner_Unknown', 0.0),
 ('occupation_partner_Professional', 0.0),
 ('occupation_partner_Retired', 0.0),
 ('occupation_partner_Manual Worker', 0.0),
 ('occupation_partner_Business Manager', 0.0),
 ('occupation_partner_Secretarial/Admin', 0.0),
 ('occupation_partner_Housewife', 0.0),
 ('occupation_partner_Other', 0.0),
 ('home_status_Own Home', 0.0),
 ('home_status_Rent from Council/HA', 0.0),
 ('home_status_Rent Privately', 0.0),
 ('home_status_Live in Parental Hom', 0.0),
 ('self_employed_No', 0.0),
 ('self_employed_partner_No', -0.2106549961345539),
 ('TVarea_Cent

# Predict Probabilities

In [24]:
logr.classes_

array([0, 1])

In [25]:
logr.predict_proba(x_train)[:,1] # probabilities for outcome being 1
# logr.predict_proba(x_train)[:,0] # probabilities for outcome being 0

array([0.99247603, 0.00870262, 0.06588989, ..., 0.81525057, 0.12035412,
       0.03045705])

In [26]:
new_data_prob=logr.predict_proba(x_new)[:,1]

# Find Cutoff to Convert Probabilities to 0/1

In [27]:
train_score=logr.predict_proba(x_train)[:,1]
real=y_train

In [28]:
cutoffs_list=np.linspace(0.001,0.999,999)

In [29]:
KS_all=[]

for cutoff in cutoffs_list:

    predicted=(train_score>cutoff).astype(int)

    TP=((real==1) & (predicted==1)).sum()
    TN=((real==0) & (predicted==0)).sum()
    FP=((real==0) & (predicted==1)).sum()
    FN=((real==1) & (predicted==0)).sum()
    P=TP+FN
    N=TN+FP

    KS=(TP/P)-(FP/N)

    KS_all.append(KS)

In [30]:
selected_cutoff=cutoffs_list[KS_all==max(KS_all)]

In [31]:
selected_cutoff
# if you get multiple values here , go with the first one

array([0.525])

In [32]:
new_data_classes=(new_data_prob>selected_cutoff).astype(int)

In [33]:
new_data_classes

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
pd.Series(new_data_classes).value_counts()

,count
0,1670
1,361
